In [ ]:
import json

from openai import OpenAI

from elasticsearch import Elasticsearch

from tqdm.auto import tqdm

In [ ]:
client = OpenAI()

In [ ]:
!curl http://localhost:9200

In [ ]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [ ]:
es_client = Elasticsearch('http://localhost:9200')

index_settings_homework = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = 'index_homework'

es_client.indices.create(index=index_name, body=index_settings_homework)

for document in tqdm(documents):
    es_client.index(index=index_name, document=document)

In [ ]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)

    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_score'])
    
    return result_docs

In [ ]:
query = "How do execute a command on a Kubernetes pod?"
elastic_search(query)


In [ ]:
def elastic_search_filter(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)

    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [ ]:
query = "How do copy a file to a Docker container?"
elastic_search_filter(query)

In [ ]:
def build_prompt(query, search_results):

    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
    Use only the facts from the CONTEXT to answer. 

    QUESTION: {question}

    CONTEXT: {context}
    """.strip()

    context = ""

    for doc in search_results:
        context += f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"    

    prompt = prompt_template.format(question=query, context=context).strip()

    return prompt

In [ ]:
query = "How do copy a file to a Docker container?"

search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

search_results = es_client.search(index=index_name, body=search_query)

In [ ]:
search_results['hits']['hits']

In [ ]:
context_template = """
Q: {question}
A: {text}
""".strip()

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [ ]:
context_pieces = []

for hit in search_results['hits']['hits']:
    doc = hit['_source']
    context_piece = context_template.format(**doc)
    context_pieces.append(context_piece)

context = '\n\n'.join(context_pieces)

In [ ]:
prompt = prompt_template.format(question=query, context=context)
len(prompt)

In [ ]:
def build_prompt(query, search_results):

    context_template = """
    Q: {question}
    A: {text}
    """.strip()

    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: {question}

    CONTEXT:
    {context}
    """.strip()    

    context_pieces = []

    for hit in search_results['hits']['hits']:
        doc = hit['_source']
        context_piece = context_template.format(**doc)
        context_pieces.append(context_piece)

    context = '\n\n'.join(context_pieces)

    prompt = prompt_template.format(question=query, context=context)

    return prompt

In [ ]:
result = build_prompt(query, search_results)
print(result)